In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from segment_anything import sam_model_registry, SamPredictor
import os


Visuals

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


Load image

In [ ]:
# Load image
image_path = "/home/s5727767/Documents/Coursework/Semester2/Masterclass/mclass_code/data/images/YiJumping/frame.001.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


Create new filename to save as

In [ ]:
# Create new filename
base, _ = os.path.splitext(os.path.basename(image_path))
output_name = f"{base}_Mask_"
output_path = os.path.join("/home/s5727767/Documents/Coursework/Semester2/Masterclass/mclass_code/data/outputs", output_name)


Show image

In [ ]:
plt.imshow(image)


Load SAM model and create Predictor

In [ ]:
# Load SAM model
checkpoint = "/home/s5727767/Documents/Coursework/Semester2/Masterclass/mclass_code/preprocessing/segmentation_models/SAM/sam_vit_h_4b8939.pth"
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=checkpoint)
sam.to("cuda" if torch.cuda.is_available() else "cpu")

predictor = SamPredictor(sam)


In [ ]:
predictor.set_image(image)


Get centre of image

In [ ]:
h, w = image.shape[:2]
centre_x = w // 2
centre_y = h // 2

input_point = np.array([[centre_x, centre_y]])
input_label = np.array([1])

Display image with star showing centre point

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
show_points(input_point, input_label, ax=plt.gca())
plt.axis('on')
plt.show()

Get masks

In [ ]:
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True
)



In [ ]:
masks.shape     # (number of masks) x H x W


Display and save various combos of masks and og image

In [ ]:
for i, (mask, score) in enumerate(zip(masks, scores)):
    # Plot
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    show_mask(mask, plt.gca())
    show_points(input_point, input_label, plt.gca())
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')


    # Create RGBA if it is the final mask
    if i == 2:
        mask_uint8 = (mask.astype(np.uint8)) * 255
        inverse_mask_uint8 = cv2.bitwise_not(mask_uint8)

        rgba = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)

        person_cutout = rgba.copy()
        person_cutout[:, :, 3] = mask_uint8
        cv2.imwrite(f"{output_path}Cutout.png", person_cutout)

        background_only = rgba.copy()
        background_only[:, :, 3] = inverse_mask_uint8
        cv2.imwrite(f"{output_path}Background.png", background_only)


    # Save
    plt.savefig(f"{output_path}{i+1}.png", bbox_inches='tight', pad_inches=0)       # Save exactly what it plots
    cv2.imwrite(f"{output_path}Only_{i+1}.png", mask.astype(np.uint8) * 255)        # Save just the mask
    # Show
    plt.show()


